<a href="https://colab.research.google.com/github/Ufuff/colab-ipynb/blob/main/Emotion_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow import lite 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import load_model

In [ ]:
def frame_from_video(dir_inp, dir_out):
    files = os.listdir(dir_inp)
    files = sorted(files)
    for f in files:
        print(f)
        vidcap = cv2.VideoCapture(dir_inp + f)
        success, image = vidcap.read()
        count = 0
        while success:
            if count < 10:
                cv2.imwrite(dir_out + f[:11] + "_frame_00%d.jpg" % count, image)
                success, image = vidcap.read()
                count += 1
            else:
                if count < 100:
                    cv2.imwrite(dir_out + f[:11] + "_frame_0%d.jpg" % count, image)
                    success, image = vidcap.read()
                    count += 1
                else:
                    cv2.imwrite(dir_out + f[:11] + "_frame_%d.jpg" % count, image)
                    success, image = vidcap.read()
                    count += 1

In [ ]:
def emotion_rec(dir_frames, dir_out_emotion_rec):
    label_map = ['Evil', 'Squeamish', 'Scared', 'Happy', 'Sad', 'Surprised', 'Neutral', 'Face_is_missing']
    filepath = "/content/drive/My Drive/models/aug/model_aug_back_best_1.h5"
    saved_model_aug = load_model(filepath)
    files = os.listdir(dir_frames)
    files = sorted(files)
    for f in files:
      face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
      img = cv2.imread(dir_frames + f, cv2.IMREAD_GRAYSCALE)
      faces = face_cascade.detectMultiScale(img, 1.3, 5)
      face_img = img[faces[0][1]:faces[0][1]+faces[0][3],faces[0][0]:faces[0][0]+faces[0][2]]
      newsize = (48, 48) 
      im = cv2.resize(face_img, newsize)
      im = im/255
      x_test_rgb = np.repeat(im[..., np.newaxis], 3, -1)
      saved_model_aug = load_model(filepath)
      scores = saved_model_aug.predict(x_test_rgb.reshape(1, 48, 48, 3))
      cv2.putText(img, label_map[scores.argmax()], (300, 500),cv2.FONT_HERSHEY_SIMPLEX, 5, (255, 255, 255), 10) 
      cv2.imwrite(dir_out_emotion_rec + f, img)

In [ ]:
!mkdir frames_2
!mkdir emo_rec_3

In [ ]:
frame_from_video("/content/drive/My Drive/project_rsl/face_component/video_2/","/content/frames_2/")

1000013_2_1_1.mp4


In [ ]:
dir_frames = "/content/frames_2/"
dir_out_emotion_rec = "/content/emo_rec_3/"
emotion_rec(dir_frames, dir_out_emotion_rec)

In [ ]:
img_array = []
files = glob.glob('/content/emo_rec_3/*.jpg')
files = sorted(files)
for filename in files:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)

out = cv2.VideoWriter('test_2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()